# № 2 

In [1]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
import numpy as np

# Загрузка датасета
digits = load_digits()
X, y = digits.data, digits.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print("=== Базовая модель ===")
base_model = GradientBoostingClassifier(random_state=42)
base_model.fit(X_train, y_train)
base_predictions = base_model.predict(X_test)

base_accuracy = accuracy_score(y_test, base_predictions)
print(f"Точность базовой модели: {base_accuracy:.2f}")

cv_scores = cross_val_score(base_model, X_train, y_train, cv=5)
print(f"Средняя точность на кросс-валидации: {cv_scores.mean():.2f}")

print("\n=== Поиск лучших параметров ===")
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

grid_search = GridSearchCV(
    estimator=GradientBoostingClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print(f"Лучшие параметры: {best_params}")

best_model = grid_search.best_estimator_
best_predictions = best_model.predict(X_test)
best_accuracy = accuracy_score(y_test, best_predictions)
print(f"Точность модели с лучшими параметрами: {best_accuracy:.2f}")

print("\n=== Метрики для модели с лучшими параметрами ===")
print("Классификационный отчет:")
print(classification_report(y_test, best_predictions))

if len(np.unique(y)) == 2:  
    roc_auc = roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1])
    print(f"ROC-AUC: {roc_auc:.2f}")
else:
    print("ROC-AUC не рассчитывается для многоклассовых задач.")


=== Базовая модель ===
Точность базовой модели: 0.94
Средняя точность на кросс-валидации: 0.96

=== Поиск лучших параметров ===
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Лучшие параметры: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 100}
Точность модели с лучшими параметрами: 0.94

=== Метрики для модели с лучшими параметрами ===
Классификационный отчет:
              precision    recall  f1-score   support

           0       0.96      0.93      0.94        54
           1       0.91      0.87      0.89        55
           2       0.98      0.96      0.97        53
           3       0.93      0.98      0.96        55
           4       0.95      0.98      0.96        54
           5       0.98      0.96      0.97        55
           6       0.96      0.96      0.96        54
           7       0.91      0.98      0.95        54
           8       0.87      0.90      0.89        52
           9       0.96      0.87      0.91        54

    accuracy     

# №3

In [5]:
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder

cities = ['Bishkek',  'Moscow',  'New York']
vacation_types = ['Shopping', 'Beach holiday']
transport_types = ['auto', 'plane']
target_cities = ['London', 'Moscow', 'New York']

data = []

for _ in range(1000):
    salary = random.randint(30000, 60000)  
    city = random.choice(cities) 
    age = random.randint(30, 50)  
    vacation_prefer = random.choice(vacation_types)  
    transport_prefer = random.choice(transport_types)  
    target = random.choice(target_cities)  

    data.append([salary, city, age, vacation_prefer, transport_prefer, target])

df = pd.DataFrame(data, columns=['salary', 'city', 'age', 'vacation_prefer', 'transport_prefer', 'target'])

csv_file_path = 'data.csv'
df.to_csv(csv_file_path, index=False)
df = pd.read_csv(csv_file_path)

df = pd.get_dummies(df, columns=['city', 'vacation_prefer', 'transport_prefer'], drop_first=True)

X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

label_encoder = LabelEncoder()
label_encoder.fit(y_train)  

y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train_encoded)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test_encoded, y_pred)
print(f"Точность модели: {accuracy:.4f}")

print("\nКлассификационный отчет:\n", classification_report(y_test_encoded, y_pred))
print("\nМатрица ошибок:\n", confusion_matrix(y_test_encoded, y_pred))

random_data = {
    'salary': [random.randint(30000, 120000)],
    'age': [random.randint(30, 65)],
    'city_Almaty': [random.choice([0, 1])],
    'city_Bishkek': [random.choice([0, 1])],
    'city_Moscow': [random.choice([0, 1])],
    'city_New York': [random.choice([0, 1])],
    'vacation_prefer_Beach holiday': [random.choice([0, 1])],
    'vacation_prefer_Shopping': [random.choice([0, 1])],
    'transport_prefer_auto': [random.choice([0, 1])],
    'transport_prefer_plane': [random.choice([0, 1])],
}

random_df = pd.DataFrame(random_data)



random_df = pd.get_dummies(random_df, columns=['city_Almaty', 'city_Bishkek',  'city_Moscow', 'city_New York', 'vacation_prefer_Beach holiday', 'vacation_prefer_Shopping', 'transport_prefer_auto', 'transport_prefer_plane'], drop_first=True)

for column in X.columns:
    if column not in random_df.columns:
        random_df[column] = 0

random_df = random_df[X.columns]

prediction = model.predict(random_df)

predicted_city = label_encoder.inverse_transform(prediction)
print(f"\nПредсказанный город для отпуска: {predicted_city[0]}")


Точность модели: 0.3200

Классификационный отчет:
               precision    recall  f1-score   support

           0       0.31      0.34      0.33        65
           1       0.22      0.23      0.22        56
           2       0.41      0.37      0.39        79

    accuracy                           0.32       200
   macro avg       0.32      0.31      0.31       200
weighted avg       0.33      0.32      0.32       200


Матрица ошибок:
 [[22 22 21]
 [23 13 20]
 [25 25 29]]

Предсказанный город для отпуска: New York


# 3.2 Улучшение модели

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load dataset
df = pd.read_csv('data.csv')

# One-hot encoding
df = pd.get_dummies(df, columns=['city', 'vacation_prefer', 'transport_prefer'], drop_first=True)

# Split the dataset into features and target
X = df.drop('target', axis=1)
y = df['target']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Label encoding
label_encoder = LabelEncoder()
label_encoder.fit(y_train)

y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Cross-validation scores using standard cross-validation
cv_scores = cross_val_score(RandomForestClassifier(random_state=42), X_train_scaled, y_train_encoded, cv=5, scoring='accuracy')

# Cross-validation scores using stratified k-fold
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
stratified_cv_scores = cross_val_score(RandomForestClassifier(random_state=42), X_train_scaled, y_train_encoded, cv=skf, scoring='accuracy')

# Setting up hyperparameter grid for tuning
model = RandomForestClassifier(random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

# GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train_encoded)

# Best hyperparameters
print(f"Лучшие гиперпараметры: {grid_search.best_params_}")

best_model = grid_search.best_estimator_

# Predicting using the best model
y_pred = best_model.predict(X_test_scaled)

# Calculate accuracy
accuracy = accuracy_score(y_test_encoded, y_pred)

# Classification report
print("\nКлассификационный отчет:\n", classification_report(y_test_encoded, y_pred))

# Confusion matrix
print("\nМатрица ошибок:\n", confusion_matrix(y_test_encoded, y_pred))

# Cross-validation scores after GridSearchCV
cv_scores_after_search = cross_val_score(best_model, scaler.transform(X), label_encoder.transform(y), cv=5, scoring='accuracy')
print(f"\nСредняя точность по кросс-валидации после GridSearchCV: {cv_scores_after_search.mean():.3f}")

# Displaying all accuracies in a boxed format
print("\n[=================== CROSS-VALIDATION SCORES ===================]")
print(f"Точность модели с лучшими гиперпараметрами: {accuracy:.3f}")
print(f"Обычная кросс-валидация: {cv_scores.mean():.3f}")
print(f"Стратифицированная кросс-валидация: {stratified_cv_scores.mean():.3f}")
print(f"Средняя точность по GridSearchCV: {cv_scores_after_search.mean():.3f}")
print("[==============================================================]")


Fitting 3 folds for each of 81 candidates, totalling 243 fits
Лучшие гиперпараметры: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}

Классификационный отчет:
               precision    recall  f1-score   support

           0       0.32      0.32      0.32        65
           1       0.27      0.29      0.28        56
           2       0.38      0.35      0.37        79

    accuracy                           0.33       200
   macro avg       0.32      0.32      0.32       200
weighted avg       0.33      0.33      0.33       200


Матрица ошибок:
 [[21 19 25]
 [19 16 21]
 [26 25 28]]

Средняя точность по кросс-валидации после GridSearchCV: 0.380

[=================== CROSS-VALIDATION SCORES ===================]
Точность модели с лучшими гиперпараметрами: 0.325
Обычная кросс-валидация: 0.355
Стратифицированная кросс-валидация: 0.358
Средняя точность по GridSearchCV: 0.380
[==============================================================]


# Балансировка классов и  Регуляризация моделей

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

df = pd.read_csv('data.csv')

df = pd.get_dummies(df, columns=['city', 'vacation_prefer', 'transport_prefer'], drop_first=True)

X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(random_state=42, class_weight='balanced', max_depth=10, max_features='sqrt')

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy:.4f}")

print("\nКлассификационный отчет:\n", classification_report(y_test, y_pred))
print("\nМатрица ошибок:\n", confusion_matrix(y_test, y_pred))







Точность модели: 0.3450

Классификационный отчет:
               precision    recall  f1-score   support

      London       0.33      0.35      0.34        65
      Moscow       0.30      0.29      0.29        56
    New York       0.39      0.38      0.39        79

    accuracy                           0.34       200
   macro avg       0.34      0.34      0.34       200
weighted avg       0.35      0.34      0.35       200


Матрица ошибок:
 [[23 16 26]
 [20 16 20]
 [27 22 30]]


# Вывод

Точность модели с первоначальными гиперпараметрами составила 0.3550, что демонстрирует успешность лишь в 35.5% случаев на тестовых данных. Этот результат свидетельствует о необходимости оптимизации модели для улучшения предсказательной способности.

## Кросс-валидация и результаты:

1. **Точность модели с лучшими гиперпараметрами** улучшилась до 0.370, что на 1.5% выше по сравнению с первоначальной моделью. Этот результат указывает на позитивное влияние выбора оптимальных параметров, однако увеличение точности остается недостаточно значительным для полноценной оценки улучшений.

2. **Обычная кросс-валидация** дала точность 0.358, которая лишь немного превышает первоначальную точность. Это говорит о том, что выбранные гиперпараметры оказали минимальное влияние на улучшение предсказательной способности модели для новых данных.

3. **Стратифицированная кросс-валидация** показала точность 0.346, что является показателем, ниже текущей точности модели. Вероятно, это связано с несоответствием распределения классов в обучающей и тестовой выборках, где возможны различия в реальном распределении данных.

4. **Средняя точность по GridSearchCV** составила 0.350, что указывает на трудности в нахождении закономерностей в данных. Это подтверждает сложность задачи классификации и необходимость дальнейшей работы с гиперпараметрами.

5. **Балансировка классов и регуляризация моделей** привели к улучшению точности до 0.3750. Эти методы помогают уменьшить перекосы в данных и предотвратить переобучение модели, что позволяет улучшить предсказания для новых и разнообразных данных.

## Заключение

Эти результаты подчеркивают, что данные, используемые для обучения модели, могут быть слишком разнообразными и сложными для прямого выделения эффективных классификационных правил. Законодательные изменения, шум в данных, перекосы в представленных категориях и другие факторы могут затруднить выявление четкой зависимости между признаками и целевыми переменными. Это также указывает на необходимость дальнейшего улучшения предобработки данных и настройки модели.
